<a href="https://colab.research.google.com/github/cinderallascent/LearnDeepLearning/blob/master/Copy_of_My_model_copy_of_pruning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


Pruning Simple Experiments
=====================================
**Author**: Brenda Zhuang_

Basic steps: 
- Set up a simple example; 
- Use multiple pruning techniques;
- Observe the behaviors from Pruned models

Requirements
------------
``"torch>=1.4.0a0+8e8a5e0"``




In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

Create a model
--------------

To compare fairly, we will be using the MNIST model for the experiment. 


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net().to(device=device)

Inspect a Module
----------------

Let's inspect the (unpruned) ``conv1`` layer in our Net model. It will contain two 
parameters ``weight`` and ``bias``, and no buffers, for now.



In [ ]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0984,  0.0411,  0.1766,  0.1884, -0.0756],
          [-0.1399,  0.0191, -0.0777,  0.1150, -0.1987],
          [ 0.0259,  0.0334, -0.1127,  0.0060, -0.1902],
          [-0.0800, -0.1647, -0.0032, -0.1455, -0.1311],
          [-0.0138, -0.0432,  0.1030, -0.1955, -0.1549]]],


        [[[-0.0998, -0.0203,  0.1126, -0.0524,  0.0367],
          [-0.0032, -0.1609, -0.0224,  0.0229,  0.1291],
          [-0.0191,  0.1257,  0.1793, -0.0510, -0.0719],
          [ 0.0388, -0.1169,  0.1236,  0.0731, -0.0336],
          [-0.0112, -0.1331, -0.0052,  0.1188,  0.0200]]],


        [[[ 0.1411,  0.0085,  0.1097, -0.0479, -0.1785],
          [-0.0797, -0.0495, -0.1180, -0.1615,  0.0116],
          [-0.1775,  0.0708,  0.1790, -0.1614,  0.0884],
          [ 0.0317, -0.1074, -0.1253,  0.0006,  0.0160],
          [-0.1220, -0.1230,  0.0395,  0.0125, -0.1751]]],


        [[[-0.1686, -0.1602,  0.0640,  0.0196, -0.1771],
          [-0.1681, -0.0297, -0.0537, -0.0

In [ ]:
print(list(module.named_buffers()))

[]


Pruning a Module
----------------

To prune a module (in this example, the ``conv1`` layer of our Net 
architecture), first select a pruning technique among those available in 
``torch.nn.utils.prune`` (or
`implement <#extending-torch-nn-utils-pruning-with-custom-pruning-functions>`_
your own by subclassing 
``BasePruningMethod``). Then, specify the module and the name of the parameter to 
prune within that module. Finally, using the adequate keyword arguments 
required by the selected pruning technique, specify the pruning parameters.

In this example, we will prune at random 30% of the connections in 
the parameter named ``weight`` in the ``conv1`` layer.
The module is passed as the first argument to the function; ``name`` 
identifies the parameter within that module using its string identifier; and 
``amount`` indicates either the percentage of connections to prune (if it 
is a float between 0. and 1.), or the absolute number of connections to 
prune (if it is a non-negative integer).



In [ ]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))

Pruning acts by removing ``weight`` from the parameters and replacing it with 
a new parameter called ``weight_orig`` (i.e. appending ``"_orig"`` to the 
initial parameter ``name``). ``weight_orig`` stores the unpruned version of 
the tensor. The ``bias`` was not pruned, so it will remain intact.



In [ ]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.0856,  0.1389,  0.0977,  0.1143,  0.0870, -0.0845, -0.1945, -0.1372,
         0.1183,  0.1799], requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0984,  0.0411,  0.1766,  0.1884, -0.0756],
          [-0.1399,  0.0191, -0.0777,  0.1150, -0.1987],
          [ 0.0259,  0.0334, -0.1127,  0.0060, -0.1902],
          [-0.0800, -0.1647, -0.0032, -0.1455, -0.1311],
          [-0.0138, -0.0432,  0.1030, -0.1955, -0.1549]]],


        [[[-0.0998, -0.0203,  0.1126, -0.0524,  0.0367],
          [-0.0032, -0.1609, -0.0224,  0.0229,  0.1291],
          [-0.0191,  0.1257,  0.1793, -0.0510, -0.0719],
          [ 0.0388, -0.1169,  0.1236,  0.0731, -0.0336],
          [-0.0112, -0.1331, -0.0052,  0.1188,  0.0200]]],


        [[[ 0.1411,  0.0085,  0.1097, -0.0479, -0.1785],
          [-0.0797, -0.0495, -0.1180, -0.1615,  0.0116],
          [-0.1775,  0.0708,  0.1790, -0.1614,  0.0884],
          [ 0.0317, -0.1074, -0.1253,  0.0006,  0.01

The pruning mask generated by the pruning technique selected above is saved 
as a module buffer named ``weight_mask`` (i.e. appending ``"_mask"`` to the 
initial parameter ``name``).



In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 1., 1., 1., 0.],
          [1., 1., 0., 0., 1.],
          [0., 1., 1., 0., 1.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 1., 0.]]],


        [[[0., 0., 1., 1., 1.],
          [1., 0., 0., 0., 0.],
          [0., 1., 0., 1., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 0., 0.],
          [1., 1., 1., 1., 1.],
          [0., 1., 0., 1., 0.],
          [0., 0., 1., 0., 0.],
          [1., 1., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 1., 1., 0., 1.],
          [1., 0., 1., 0., 0.],
          [1., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]]],


        [[[0., 1., 0., 0., 1.],
          [0., 1., 0., 1., 1.],
          [0., 1., 1., 0., 0.],
          [0., 0., 0., 0., 1.],
          [1., 1., 0., 0., 1.]]],


        [[[0., 1., 1., 0., 1.],
          [0., 1., 1., 1., 1.],
          [0., 1., 1., 0., 0.],
          [1., 1., 1., 0., 1.],
          [0., 1., 0., 0., 1.]]],



For the forward pass to work without modification, the ``weight`` attribute 
needs to exist. The pruning techniques implemented in 
``torch.nn.utils.prune`` compute the pruned version of the weight (by 
combining the mask with the original parameter) and store them in the 
attribute ``weight``. Note, this is no longer a parameter of the ``module``,
it is now simply an attribute.



In [ ]:
print(module.weight)

tensor([[[[ 0.0000,  0.0411,  0.1766,  0.1884, -0.0000],
          [-0.1399,  0.0191, -0.0000,  0.0000, -0.1987],
          [ 0.0000,  0.0334, -0.1127,  0.0000, -0.1902],
          [-0.0000, -0.0000, -0.0032, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.1030, -0.1955, -0.0000]]],


        [[[-0.0000, -0.0000,  0.1126, -0.0524,  0.0367],
          [-0.0032, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000,  0.1257,  0.0000, -0.0510, -0.0000],
          [ 0.0000, -0.1169,  0.0000,  0.0000, -0.0336],
          [-0.0112, -0.1331, -0.0052,  0.1188,  0.0200]]],


        [[[ 0.0000,  0.0085,  0.1097, -0.0000, -0.0000],
          [-0.0797, -0.0495, -0.1180, -0.1615,  0.0116],
          [-0.0000,  0.0708,  0.0000, -0.1614,  0.0000],
          [ 0.0000, -0.0000, -0.1253,  0.0000,  0.0000],
          [-0.1220, -0.1230,  0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0297, -0.0537, -0.0000,  0.0987],
          [ 0.1338,

Finally, pruning is applied prior to each forward pass using PyTorch's
``forward_pre_hooks``. Specifically, when the ``module`` is pruned, as we 
have done here, it will acquire a ``forward_pre_hook`` for each parameter 
associated with it that gets pruned. In this case, since we have so far 
only pruned the original parameter named ``weight``, only one hook will be
present.



In [ ]:
print(module._forward_pre_hooks)

OrderedDict([(1, <torch.nn.utils.prune.PruningContainer object at 0x7fc9bd29acd0>)])


For completeness, we can now prune the ``bias`` too, to see how the 
parameters, buffers, hooks, and attributes of the ``module`` change.
Just for the sake of trying out another pruning technique, here we prune the 
3 smallest entries in the bias by L1 norm, as implemented in the 
``l1_unstructured`` pruning function.



In [ ]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))

We now expect the named parameters to include both ``weight_orig`` (from 
before) and ``bias_orig``. The buffers will include ``weight_mask`` and 
``bias_mask``. The pruned versions of the two tensors will exist as 
module attributes, and the module will now have two ``forward_pre_hooks``.



In [ ]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.0984,  0.0411,  0.1766,  0.1884, -0.0756],
          [-0.1399,  0.0191, -0.0777,  0.1150, -0.1987],
          [ 0.0259,  0.0334, -0.1127,  0.0060, -0.1902],
          [-0.0800, -0.1647, -0.0032, -0.1455, -0.1311],
          [-0.0138, -0.0432,  0.1030, -0.1955, -0.1549]]],


        [[[-0.0998, -0.0203,  0.1126, -0.0524,  0.0367],
          [-0.0032, -0.1609, -0.0224,  0.0229,  0.1291],
          [-0.0191,  0.1257,  0.1793, -0.0510, -0.0719],
          [ 0.0388, -0.1169,  0.1236,  0.0731, -0.0336],
          [-0.0112, -0.1331, -0.0052,  0.1188,  0.0200]]],


        [[[ 0.1411,  0.0085,  0.1097, -0.0479, -0.1785],
          [-0.0797, -0.0495, -0.1180, -0.1615,  0.0116],
          [-0.1775,  0.0708,  0.1790, -0.1614,  0.0884],
          [ 0.0317, -0.1074, -0.1253,  0.0006,  0.0160],
          [-0.1220, -0.1230,  0.0395,  0.0125, -0.1751]]],


        [[[-0.1686, -0.1602,  0.0640,  0.0196, -0.1771],
          [-0.1681, -0.0297, -0.0537,

In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 1., 1., 1., 0.],
          [1., 1., 0., 0., 1.],
          [0., 1., 1., 0., 1.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 1., 0.]]],


        [[[0., 0., 1., 1., 1.],
          [1., 0., 0., 0., 0.],
          [0., 1., 0., 1., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 0., 0.],
          [1., 1., 1., 1., 1.],
          [0., 1., 0., 1., 0.],
          [0., 0., 1., 0., 0.],
          [1., 1., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 1., 1., 0., 1.],
          [1., 0., 1., 0., 0.],
          [1., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]]],


        [[[0., 1., 0., 0., 1.],
          [0., 1., 0., 1., 1.],
          [0., 1., 1., 0., 0.],
          [0., 0., 0., 0., 1.],
          [1., 1., 0., 0., 1.]]],


        [[[0., 1., 1., 0., 1.],
          [0., 1., 1., 1., 1.],
          [0., 1., 1., 0., 0.],
          [1., 1., 1., 0., 1.],
          [0., 1., 0., 0., 1.]]],



In [ ]:
print(module.bias)

tensor([ 0.0000,  0.1389,  0.0977,  0.1143,  0.0000, -0.0000, -0.1945, -0.1372,
         0.1183,  0.1799], grad_fn=<MulBackward0>)


In [ ]:
print(module._forward_pre_hooks)

OrderedDict([(1, <torch.nn.utils.prune.PruningContainer object at 0x7fc9bd29acd0>), (2, <torch.nn.utils.prune.L1Unstructured object at 0x7fc9bd22a0d0>)])


Iterative Pruning
-----------------

The same parameter in a module can be pruned multiple times, with the 
effect of the various pruning calls being equal to the combination of the
various masks applied in series.
The combination of a new mask with the old mask is handled by the 
``PruningContainer``'s ``compute_mask`` method.

Say, for example, that we now want to further prune ``module.weight``, this
time using structured pruning along the 0th axis of the tensor (the 0th axis 
corresponds to the output channels of the convolutional layer and has 
dimensionality 6 for ``conv1``), based on the channels' L2 norm. This can be 
achieved using the ``ln_structured`` function, with ``n=2`` and ``dim=0``.



In [ ]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to 
# 50% (3 out of 6) of the channels, while preserving the action of the 
# previous mask.
print(module.weight)

tensor([[[[ 0.0000,  0.0411,  0.1766,  0.1884, -0.0000],
          [-0.1399,  0.0191, -0.0000,  0.0000, -0.1987],
          [ 0.0000,  0.0334, -0.1127,  0.0000, -0.1902],
          [-0.0000, -0.0000, -0.0032, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.1030, -0.1955, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,

The corresponding hook will now be of type 
``torch.nn.utils.prune.PruningContainer``, and will store the history of 
pruning applied to the ``weight`` parameter.



In [ ]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))  # pruning history in the container

[<torch.nn.utils.prune.RandomUnstructured object at 0x7fca191ed210>, <torch.nn.utils.prune.RandomUnstructured object at 0x7fc9bd29aad0>, <torch.nn.utils.prune.LnStructured object at 0x7fc9bd29af50>]


Serializing a pruned model
--------------------------
All relevant tensors, including the mask buffers and the original parameters
used to compute the pruned tensors are stored in the model's ``state_dict`` 
and can therefore be easily serialized and saved, if needed.



In [ ]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


Remove pruning re-parametrization
---------------------------------

To make the pruning permanent, remove the re-parametrization in terms
of ``weight_orig`` and ``weight_mask``, and remove the ``forward_pre_hook``,
we can use the ``remove`` functionality from ``torch.nn.utils.prune``.
Note that this doesn't undo the pruning, as if it never happened. It simply 
makes it permanent, instead, by reassigning the parameter ``weight`` to the 
model parameters, in its pruned version.



Prior to removing the re-parametrization:



In [ ]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.0984,  0.0411,  0.1766,  0.1884, -0.0756],
          [-0.1399,  0.0191, -0.0777,  0.1150, -0.1987],
          [ 0.0259,  0.0334, -0.1127,  0.0060, -0.1902],
          [-0.0800, -0.1647, -0.0032, -0.1455, -0.1311],
          [-0.0138, -0.0432,  0.1030, -0.1955, -0.1549]]],


        [[[-0.0998, -0.0203,  0.1126, -0.0524,  0.0367],
          [-0.0032, -0.1609, -0.0224,  0.0229,  0.1291],
          [-0.0191,  0.1257,  0.1793, -0.0510, -0.0719],
          [ 0.0388, -0.1169,  0.1236,  0.0731, -0.0336],
          [-0.0112, -0.1331, -0.0052,  0.1188,  0.0200]]],


        [[[ 0.1411,  0.0085,  0.1097, -0.0479, -0.1785],
          [-0.0797, -0.0495, -0.1180, -0.1615,  0.0116],
          [-0.1775,  0.0708,  0.1790, -0.1614,  0.0884],
          [ 0.0317, -0.1074, -0.1253,  0.0006,  0.0160],
          [-0.1220, -0.1230,  0.0395,  0.0125, -0.1751]]],


        [[[-0.1686, -0.1602,  0.0640,  0.0196, -0.1771],
          [-0.1681, -0.0297, -0.0537,

In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 1., 1., 1., 0.],
          [1., 1., 0., 0., 1.],
          [0., 1., 1., 0., 1.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 1., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 1., 0., 0., 1.],
          [0., 1., 0., 1., 1.],
          [0., 1., 1., 0., 0.],
          [0., 0., 0., 0., 1.],
          [1., 1., 0., 0., 1.]]],


        [[[0., 1., 1., 0., 1.],
          [0., 1., 1., 1., 1.],
          [0., 1., 1., 0., 0.],
          [1., 1., 1., 0., 1.],
          [0., 1., 0., 0., 1.]]],



In [ ]:
print(module.weight)

tensor([[[[ 0.0000,  0.0411,  0.1766,  0.1884, -0.0000],
          [-0.1399,  0.0191, -0.0000,  0.0000, -0.1987],
          [ 0.0000,  0.0334, -0.1127,  0.0000, -0.1902],
          [-0.0000, -0.0000, -0.0032, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.1030, -0.1955, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,

After removing the re-parametrization:



In [ ]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([ 0.0856,  0.1389,  0.0977,  0.1143,  0.0870, -0.0845, -0.1945, -0.1372,
         0.1183,  0.1799], requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000,  0.0411,  0.1766,  0.1884, -0.0000],
          [-0.1399,  0.0191, -0.0000,  0.0000, -0.1987],
          [ 0.0000,  0.0334, -0.1127,  0.0000, -0.1902],
          [-0.0000, -0.0000, -0.0032, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.1030, -0.1955, -0.0000]]],


        [[[-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000]]],


        [[[ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000,  0.0000,  0.00

In [ ]:
print(list(module.named_buffers()))

[('bias_mask', tensor([0., 1., 1., 1., 0., 0., 1., 1., 1., 1.]))]


Pruning multiple parameters in a model 
--------------------------------------

By specifying the desired pruning technique and parameters, we can easily 
prune multiple tensors in a network, perhaps according to their type, as we 
will see in this example.



In [ ]:
import torchvision
batch_size_test = 1000
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape
test_losses = []

In [ ]:
new_model = Net()
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers 
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.4)
    # prune 40% of connections in all linear layers 
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.1)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask'])


In [ ]:
def test():
  new_model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = new_model(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.2964, Accuracy: 1383/10000 (14%)



Global pruning
--------------

So far, we only looked at what is usually referred to as "local" pruning,
i.e. the practice of pruning tensors in a model one by one, by 
comparing the statistics (weight magnitude, activation, gradient, etc.) of 
each entry exclusively to the other entries in that tensor. However, a 
common and perhaps more powerful technique is to prune the model all at 
once, by removing (for example) the lowest 20% of connections across the 
whole model, instead of removing the lowest 20% of connections in each 
layer. This is likely to result in different pruning percentages per layer.
Let's see how to do that using ``global_unstructured`` from 
``torch.nn.utils.prune``.



In [ ]:
model = Net()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

Now we can check the sparsity induced in every pruned parameter, which will 
not be equal to 20% in each layer. However, the global sparsity will be 
(approximately) 20%.



In [ ]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 6.40%
Sparsity in conv2.weight: 17.82%
Sparsity in fc1.weight: 21.31%
Sparsity in fc2.weight: 6.80%
Global sparsity: 20.00%


In [ ]:
new_model = model
test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3090, Accuracy: 931/10000 (9%)



Extending ``torch.nn.utils.prune`` with custom pruning functions
------------------------------------------------------------------
To implement your own pruning function, you can extend the
``nn.utils.prune`` module by subclassing the ``BasePruningMethod``
base class, the same way all other pruning methods do. The base class
implements the following methods for you: ``__call__``, ``apply_mask``,
``apply``, ``prune``, and ``remove``. Beyond some special cases, you shouldn't
have to reimplement these methods for your new pruning technique.
You will, however, have to implement ``__init__`` (the constructor),
and ``compute_mask`` (the instructions on how to compute the mask
for the given tensor according to the logic of your pruning
technique). In addition, you will have to specify which type of
pruning this technique implements (supported options are ``global``,
``structured``, and ``unstructured``). This is needed to determine
how to combine masks in the case in which pruning is applied
iteratively. In other words, when pruning a pre-pruned parameter,
the current prunining techique is expected to act on the unpruned
portion of the parameter. Specifying the ``PRUNING_TYPE`` will
enable the ``PruningContainer`` (which handles the iterative
application of pruning masks) to correctly identify the slice of the
parameter to prune.

Let's assume, for example, that you want to implement a pruning
technique that prunes every other entry in a tensor (or -- if the
tensor has previously been pruned -- in the remaining unpruned
portion of the tensor). This will be of ``PRUNING_TYPE='unstructured'``
because it acts on individual connections in a layer and not on entire
units/channels (``'structured'``), or across different parameters
(``'global'``).



In [ ]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0 
        return mask

Now, to apply this to a parameter in an ``nn.Module``, you should
also provide a simple function that instantiates the method and
applies it.



In [ ]:
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module) 
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the 
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the 
    original (unpruned) parameter is stored in a new parameter named 
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module
    
    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

Let's try it out!



In [ ]:
model = Net()
foobar_unstructured(model.fc2, name='bias')

print(model.fc2.bias_mask)

In [ ]:
new_model = model
test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3150, Accuracy: 1169/10000 (12%)

